In [37]:
import numpy as np
import pandas as pd
from surprise import Dataset, Reader, KNNBasic, KNNWithZScore
from surprise.model_selection import train_test_split
from surprise import accuracy

# Carregando o conjunto de dados MovieLens
data = Dataset.load_builtin('ml-100k')

# Definindo o leitor de dados
reader = Reader(rating_scale=(1, 5))

# Carregando o dataset em forma de dados brutos (não apenas o 'trainset')
df = data.build_full_trainset()

# Dividindo o conjunto de dados em treinamento e teste
trainset, testset = train_test_split(data, test_size=0.2)  # Aqui estamos passando 'data', não 'df'

# Algoritmo baseado em usuário (KNN com Z-score)
sim_options_user = {
    'name': 'pearson',
    'user_based': True  # Recomendação baseada no usuário
}
model_user = KNNWithZScore(sim_options=sim_options_user)

# Treinando o modelo baseado em usuário
model_user.fit(trainset)

# Fazendo previsões no conjunto de teste
predictions_user = model_user.test(testset)

# Calculando o erro quadrático médio (RMSE) para o modelo baseado em usuário
rmse_user = accuracy.rmse(predictions_user)
print(f'RMSE para o modelo baseado em usuário: {rmse_user:.4f}')

# Algoritmo baseado em item (KNN básico)
sim_options_item = {
    'name': 'pearson',
    'user_based': False  # Recomendação baseada no item
}
model_item = KNNBasic(sim_options=sim_options_item)

# Treinando o modelo baseado em item
model_item.fit(trainset)

# Fazendo previsões no conjunto de teste
predictions_item = model_item.test(testset)

# Calculando o erro quadrático médio (RMSE) para o modelo baseado em item
rmse_item = accuracy.rmse(predictions_item)
print(f'RMSE para o modelo baseado em item: {rmse_item:.4f}')

# --------------------------
# EXIBIR FILMES JÁ AVALIADOS PELO USUÁRIO
# --------------------------

# Encontrar um usuário válido do conjunto de treino
user_inner_id = np.random.choice(trainset.all_users())  # Seleciona um ID interno aleatório do conjunto de treino
user_raw_id = trainset.to_raw_uid(user_inner_id)  # Converte o ID interno para o ID original

# Obtendo todas as avaliações já feitas pelo usuário
user_ratings = [(trainset.to_raw_iid(iid), r_ui) for (iid, r_ui) in trainset.ur[user_inner_id]]

# Carregar os nomes dos filmes do arquivo de metadados 'u.item'
movie_names_df = pd.read_csv('ml-100k/u.item', sep='|', header=None, encoding='latin-1', usecols=[0, 1], names=['movie_id', 'movie_name'])
movie_names_df['movie_id'] = movie_names_df['movie_id'].astype(int)

# Exibindo os filmes já avaliados pelo usuário com nomes
print(f'\nFilmes já avaliados pelo usuário {user_raw_id}:')
for movie_id, rating in user_ratings:
    movie_name = movie_names_df[movie_names_df['movie_id'] == int(movie_id)]['movie_name'].values[0]
    print(f'Filme: {movie_name} - Avaliação: {rating}')

# --------------------------
# RECOMENDAR FILMES NÃO AVALIADOS
# --------------------------

# Pegando todas as avaliações possíveis que o usuário ainda não avaliou
predictions = [pred for pred in predictions_user if pred.uid == user_raw_id]
predictions.sort(key=lambda x: x.est, reverse=True)

# Selecionando os top N filmes recomendados
top_n = 5
top_recommendations = predictions[:top_n]
print(f'\nTop {top_n} recomendações para o usuário {user_raw_id}:')
for i, pred in enumerate(top_recommendations, 1):
    movie_name = movie_names_df[movie_names_df['movie_id'] == int(pred.iid)]['movie_name'].values[0]
    print(f'{i}: {movie_name} com estimativa de avaliação: {pred.est:.2f}')


Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9452
RMSE para o modelo baseado em usuário: 0.9452
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.0450
RMSE para o modelo baseado em item: 1.0450

Filmes já avaliados pelo usuário 98:
Filme: Birdcage, The (1996) - Avaliação: 5.0
Filme: Ruling Class, The (1972) - Avaliação: 3.0
Filme: Annie Hall (1977) - Avaliação: 5.0
Filme: Cold Comfort Farm (1995) - Avaliação: 5.0
Filme: Ed Wood (1994) - Avaliação: 4.0
Filme: Monty Python and the Holy Grail (1974) - Avaliação: 2.0
Filme: Return of the Pink Panther, The (1974) - Avaliação: 3.0
Filme: Sleeper (1973) - Avaliação: 3.0
Filme: Murder at 1600 (1997) - Avaliação: 3.0
Filme: Cool Hand Luke (1967) - Avaliação: 5.0
Filme: Victor/Victoria (1982) - Avaliação: 5.0
Filme: Butch Cassidy and the Sundance Kid (1969) - Avaliação: 5.0
Filme: Stand by Me (1986) - Avaliação: 3.0
Filme: Beautician and the Beast, The (1997) - Avaliação: